In [1]:
!pip install qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 48.0 MB/s eta 0:00:00:00:0100:01


In [2]:
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq

model_id = '/kaggle/input/new-cat1-mod/LLaMA-Factory/qwen2vl_2b_instruct_lora_merged_cat1_new'


model = AutoModelForVision2Seq.from_pretrained(
  model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  # attn_implementation="flash_attention_2",
)
processor = AutoProcessor.from_pretrained(model_id)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import pandas as pd


test_df = pd.read_csv('/kaggle/input/mesho-chll/MESHO/test.csv')


Men_Tshirts_df = test_df[test_df['Category'] == 'Men Tshirts']


Men_Tshirts_df['image'] = Men_Tshirts_df['id'].apply(
    lambda x: f"/kaggle/input/mesho-chll/MESHO/test_images/{str(x).zfill(6)}.jpg"
)


Men_Tshirts_df.to_csv('Men_Tshirts_df_cat1.csv', index=False)


/tmp/ipykernel_30/1364480072.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Women_Tshirts_df['image'] = Women_Tshirts_df['id'].apply(


In [4]:
men_df = pd.read_csv('/kaggle/working/Men_Tshirts_df_cat1.csv')

In [5]:
men_df

,id,Category,image
0,0,Men Tshirts,/kaggle/input/mesho-chll/MESHO/test_images/000...
1,1,Men Tshirts,/kaggle/input/mesho-chll/MESHO/test_images/000...
2,2,Men Tshirts,/kaggle/input/mesho-chll/MESHO/test_images/000...
3,3,Men Tshirts,/kaggle/input/mesho-chll/MESHO/test_images/000...
4,4,Men Tshirts,/kaggle/input/mesho-chll/MESHO/test_images/000...
...,...,...,...
3782,3782,Men Tshirts,/kaggle/input/mesho-chll/MESHO/test_images/003...
3783,3783,Men Tshirts,/kaggle/input/mesho-chll/MESHO/test_images/003...
3784,3784,Men Tshirts,/kaggle/input/mesho-chll/MESHO/test_images/003...
3785,3785,Men Tshirts,/kaggle/input/mesho-chll/MESHO/test_images/003...


In [7]:
from qwen_vl_utils import process_vision_info
def generate_description(sample, model, processor):
    prompt = """Given the provided product category (##CATEGORY##) and its corresponding image, please predict the following attributes for the product: attr_1 to attr_10 based on previously observed patterns and examples. Ensure that your predictions are accurate and relevant to the visual characteristics in the image.

    ##CATEGORY##: {category}
    """
    
            
         
    messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt.format(category=sample["Category"]),
                    },
                    {
                        "type": "image",
                        "image": sample["image"],
                    }
                ],
            },
           
        ]
   
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(model.device)

    generated_ids = model.generate(**inputs, max_new_tokens=256, top_p=1.0, do_sample=True, temperature=0.8)
    generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return output_text[0]




In [ ]:
from tqdm import tqdm
import pandas as pd


results = []


for index, row in tqdm(men_df.iterrows(), total=men_df.shape[0]):
    try:
       
        description_text = generate_description(row, model, processor)
        
        
        description_dict = eval(description_text)  
        result = {
            "id": row["id"],
            "Category": row["Category"]
        }
        
       
        attr_count = 0  
        for i in range(1, 11):
            attr_key = f"attr_{i}"
            attr_value = description_dict.get(attr_key, "")
            result[attr_key] = attr_value
            if attr_value.lower() != "no":
                attr_count += 1  
        
        
        result["len"] = attr_count
        results.append(result)
    
    except Exception as e:
        print(f"Error processing id {row['id']}: {e}")

results_df = pd.DataFrame(results)
results_df.to_csv('cat1_new.csv', index=False)


  1%|          | 29/3787 [02:29<5:12:36,  4.99s/it]